# Keras Tuner

In this notebook, we turn to hyperparameter tuning using Keras Tuner.  Our objective is to find better hyperparams and network architecture. Although we will not spend too much time on network architecture, you probably will get enough information to try on your own.

@Date: 13-Sep-2020 | @Author: Katnoria

In [1]:
import kerastuner as kt
import tensorflow as tf
import tensorflow_datasets as tfds
import IPython

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

In [3]:
def build_model(hp):
    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = inputs
    for i in range(hp.Int('conv_blocks', 3, 5, default=3)):
        filters = hp.Int('filters_' + str(i), 32, 256, step=32)
        for _ in range(2):
            x = tf.keras.layers.Convolution2D(
              filters, kernel_size=(3, 3), padding='same')(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.ReLU()(x)
        if hp.Choice('pooling_' + str(i), ['avg', 'max']) == 'max':
            x = tf.keras.layers.MaxPool2D()(x)
        else:
            x = tf.keras.layers.AvgPool2D()(x)
    x = tf.keras.layers.GlobalAvgPool2D()(x)
    x = tf.keras.layers.Dense(
      hp.Int('hidden_size', 30, 100, step=10, default=50),
      activation='relu')(x)
    x = tf.keras.layers.Dropout(
      hp.Float('dropout', 0, 0.5, step=0.1, default=0.5))(x)
    outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(
      optimizer=tf.keras.optimizers.Adam(
        hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'])
    return model

In [4]:
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2)

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json


In [5]:
data = tfds.load('cifar10')
train_ds, test_ds = data['train'], data['test']


def standardize_record(record):
    return tf.cast(record['image'], tf.float32) / 255., record['label']

In [6]:
train_ds = train_ds.map(standardize_record).cache().batch(64).shuffle(10000)
test_ds = test_ds.map(standardize_record).cache().batch(64)

In [7]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [8]:
tuner.search(train_ds,
             validation_data=test_ds,
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1), ClearTrainingOutput()])

Epoch 1/2
191/782 [======>.......................] - ETA: 0s - loss: 2.3206 - accuracy: 0.10 - ETA: 4:33 - loss: 2.3002 - accuracy: 0.10 - ETA: 6:04 - loss: 2.2969 - accuracy: 0.10 - ETA: 6:48 - loss: 2.2951 - accuracy: 0.10 - ETA: 7:16 - loss: 2.2903 - accuracy: 0.10 - ETA: 7:33 - loss: 2.2817 - accuracy: 0.12 - ETA: 7:45 - loss: 2.2724 - accuracy: 0.12 - ETA: 7:52 - loss: 2.2679 - accuracy: 0.12 - ETA: 7:56 - loss: 2.2615 - accuracy: 0.13 - ETA: 7:59 - loss: 2.2532 - accuracy: 0.13 - ETA: 8:04 - loss: 2.2451 - accuracy: 0.14 - ETA: 8:05 - loss: 2.2382 - accuracy: 0.15 - ETA: 8:06 - loss: 2.2377 - accuracy: 0.15 - ETA: 8:07 - loss: 2.2354 - accuracy: 0.16 - ETA: 8:08 - loss: 2.2316 - accuracy: 0.16 - ETA: 8:09 - loss: 2.2264 - accuracy: 0.17 - ETA: 8:11 - loss: 2.2198 - accuracy: 0.17 - ETA: 8:12 - loss: 2.2161 - accuracy: 0.18 - ETA: 8:12 - loss: 2.2142 - accuracy: 0.18 - ETA: 8:12 - loss: 2.2089 - accuracy: 0.18 - ETA: 8:13 - loss: 2.2029 - accuracy: 0.18 - ETA: 8:14 - loss: 2.1978 

KeyboardInterrupt: 

In [ ]:
best_model = tuner.get_best_models(1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]

In [ ]:
best_model.evaluate(test_ds)

In [ ]:
# save
path = "models/cifar10-kt"
best_model.save_weights(path)

In [ ]:
best_hyperparameters.values